In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from project_helpers import *
from project_helpers import get_dataset_info
from cv2 import SIFT_create, cvtColor, COLOR_RGB2GRAY, FlannBasedMatcher, drawMatchesKnn
rgb2gray = lambda img: cvtColor(img, COLOR_RGB2GRAY)
sift = SIFT_create(contrastThreshold=0.02, edgeThreshold=10, nOctaveLayers=3)

Extracting data

In [2]:
def read_images(img_names):
  images = []
  for name in img_names:
    images.append((plt.imread(name) * 255).astype('uint8'))
  return images

Getting SIFT points and descriptors

In [3]:
def extract_sift_features(images):
  keypoints = []
  descriptors = []
  for image in images:
    kp, des = sift.detectAndCompute(rgb2gray(image),None)
    keypoints.append(kp)
    descriptors.append(des)
    #print(len(kp))
  return keypoints, descriptors

#images = read_images(img_names)
#extract_sift_features(images)

Matching sift points and descriptors

In [4]:
def match_sift_features(keypoints, descriptors):
  keypoint_pairs = []
  
  for i in range(len(keypoints)-1):
    all_matches = FlannBasedMatcher().knnMatch(descriptors[i], descriptors[i+1], k=2)
    print('all matches', len(all_matches))
    good_matches = []
    for m,n in all_matches:
        if m.distance < 0.75*n.distance:
            good_matches.append([m])
    print('god matches', len(good_matches), '\n ')
    
    kp1 = keypoints[i]
    kp2 = keypoints[i+1]
            
    x1 = np.array([[kp1[match[0].queryIdx].pt[0], kp1[match[0].queryIdx].pt[1]] for match in good_matches])
    x2 = np.array([[kp2[match[0].trainIdx].pt[0], kp2[match[0].trainIdx].pt[1]] for match in good_matches])
    x1_homo = np.vstack((x1.T, np.ones(x1.shape[0])))
    x2_homo = np.vstack((x2.T, np.ones(x2.shape[0])))
    
    keypoint_pairs.append([x1_homo,x2_homo])
    
  return keypoint_pairs

In [5]:
# def match_sift_features(keypoints, descriptors):
#   keypoint_pairs = []
  
#   for i in range(len(keypoints)-1):
#     all_matches = FlannBasedMatcher().knnMatch(descriptors[i], descriptors[i+1], k=2)
#     good_matches = []
#     for m,n in all_matches:
#         if m.distance < 0.75*n.distance:
#             good_matches.append([m])
    
#     kp1 = keypoints[i]
#     kp2 = keypoints[i+1]
            
#     x1 = np.array([[kp1[match[0].queryIdx].pt[0], kp1[match[0].queryIdx].pt[1]] for match in good_matches])
#     x2 = np.array([[kp2[match[0].trainIdx].pt[0], kp2[match[0].trainIdx].pt[1]] for match in good_matches])
#     x1_homo = np.vstack((x1.T, np.ones(x1.shape[0])))
#     x2_homo = np.vstack((x2.T, np.ones(x2.shape[0])))
    
#     keypoint_pairs.append([x1_homo,x2_homo])
    
#   return keypoint_pairs

Estimating E robustly and extracting P fro E

In [6]:
def get_camera_matrices(keypoint_pairs, K, pixel_threshold):
  eps = pixel_threshold / K[0,0]
  
  camera_matrices = []
  keypoint_pairs_inlier = []
  
  P1_k = np.array([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1, 0]])
  
  for pair in keypoint_pairs:
    x1, x2 = pair
    x1_k = np.linalg.inv(K) @ x1
    x2_k = np.linalg.inv(K) @ x2
    
    E_robust, inliers, errs, iters = estimate_E_robust(x1_k, x2_k, eps)
    #print(E_robust)
    x1_inliers = inliers[:3]
    x2_inliers = inliers[3:]
    keypoint_pairs_inlier.append([x1_inliers,x2_inliers])
    
    P2_extract = extract_P_from_E(E_robust)
    
    best_i = None
    best_count = 0

    for i in range(4):
      count = count_points_in_front(P1_k, P2_extract[i], x1_k, x2_k)
      print('Came',i,'inliers',count)
      if count > best_count:
          best_count = count
          best_i = i
    print(' ')
    best_p2 = P2_extract[best_i]
    camera_matrices.append(best_p2)
    
  return camera_matrices, keypoint_pairs_inlier

In [7]:
def chain_camera_rotation(camera_matrices):
  R0 = camera_matrices[0][:,:-1]
  absolute_rotations = [R0]
  for i in range(len(camera_matrices)-1):
    R1 = camera_matrices[i+1][:,:-1]
    #print(R1)
    #print(R0)
    print('prutt <3')
    absolute_rotations.append(absolute_rotations[i] @ R1)
  print()
  return absolute_rotations

# P1_k = np.array([[1, 0, 0, 0],
#               [0, 1, 0, 0],
#               [0, 0, 1, 0]])
# print(P1_k[:,:-1])

In [ ]:
def estimate_camera_DLT(x, Xmodel):
    if x.shape[0] != 3:
        raise ValueError("x must be of shape 3xN")
    
    if Xmodel.shape[0] != 4:
        raise ValueError("Xmodel must be of shape 4xN")
    
    x_h = x.copy()
    Xmodel_h = Xmodel.copy()
    
    n = x_h.shape[1]
    
    M = np.zeros((2*n, 12), dtype=float)
    
    for i in range(n):
        u = x_h[0, i]
        v = x_h[1, i]
        X = Xmodel_h[:, i]
        
        M[2*i, 0:4] = X
        M[2*i, 8:12] = -u*X
        
        M[2*i+1, 4:8] = X
        M[2*i+1, 8:12] = -v*X
        
    U, S, Vt = np.linalg.svd(M)
    
    v = Vt[-1]
    
    P = v.reshape((3, 4))
    
    P /= P[-1, -1]
    
    
    print(np.linalg.norm(M @ v))
    print(S[-1])
    
    return P


In [8]:
#def run_sfm(dataset_num):
K, img_names, init_pair, pixel_threshold = get_dataset_info(2)

print('image names', img_names)

images = read_images(img_names)
print('images_len',len(images))

#print('number of images', len(images))
#print('image shape', images[0].shape)  

keypoints, descriptors = extract_sift_features(images)

keypoint_pair_matches = match_sift_features(keypoints, descriptors)
#print(len(keypoint_pairs))

camera_matrices, keypoint_pairs_inlier = get_camera_matrices(keypoint_pair_matches, K, pixel_threshold)

#print(camera_matrices)

camera_matrices_chained = chain_camera_rotation(camera_matrices)



#run_sfm(2)

image names ['data/2/DSC_0025.JPG', 'data/2/DSC_0026.JPG', 'data/2/DSC_0027.JPG', 'data/2/DSC_0028.JPG', 'data/2/DSC_0029.JPG', 'data/2/DSC_0030.JPG', 'data/2/DSC_0031.JPG', 'data/2/DSC_0032.JPG', 'data/2/DSC_0033.JPG']
images_len 9
all matches 29650
god matches 14739 
 
all matches 31595
god matches 15973 
 
all matches 30971
god matches 15260 
 
all matches 29877
god matches 15338 
 
all matches 30707
god matches 15584 
 
all matches 31014
god matches 15575 
 
all matches 32028
god matches 15324 
 
all matches 31646
god matches 14413 
 
Came 0 inliers 0
Came 1 inliers 0
Came 2 inliers 43
Came 3 inliers 14696
 
Came 0 inliers 64
Came 1 inliers 15909
Came 2 inliers 0
Came 3 inliers 0
 
Came 0 inliers 41
Came 1 inliers 15219
Came 2 inliers 0
Came 3 inliers 0
 
Came 0 inliers 0
Came 1 inliers 0
Came 2 inliers 50
Came 3 inliers 15288
 
Came 0 inliers 15538
Came 1 inliers 46
Came 2 inliers 0
Came 3 inliers 0
 
Came 0 inliers 15524
Came 1 inliers 51
Came 2 inliers 0
Came 3 inliers 0
 
Came 

In [9]:
print(init_pair[0])
print(len(keypoints))
print(keypoints[init_pair[0]])

init_pair_keypoints = [keypoints[init_pair[0]],keypoints[init_pair[1]]]
init_pair_descriptors = [descriptors[init_pair[0]],descriptors[init_pair[1]]]

print(len(init_pair_descriptors))

keypoint_pair_init = match_sift_features(init_pair_keypoints, init_pair_descriptors)

camera_matrices_init, keypoint_pairs_init_inlier = get_camera_matrices(keypoint_pair_init, K, pixel_threshold)

print(camera_matrices_init[0].shape)

P1 = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0]])
print(keypoint_pairs_init_inlier[0][0].shape)
print(keypoint_pairs_init_inlier[0][1].shape)

n = len(keypoint_pairs_init_inlier[0][0])
three_dee_poynts = []
for i in range(n):
        X_init = triangulate_3D_point_DLT(keypoint_pairs_init_inlier[0][0][:, i], keypoint_pairs_init_inlier[0][1][:, i], P1, camera_matrices_init[0])
        three_dee_poynts.append(X_init)


0
9
(< cv2.KeyPoint 0000020CFFF25020>, < cv2.KeyPoint 0000020C98D0BD50>, < cv2.KeyPoint 0000020CFF98E6A0>, < cv2.KeyPoint 0000020C98263C00>, < cv2.KeyPoint 0000020C98263BD0>, < cv2.KeyPoint 0000020C98CD8060>, < cv2.KeyPoint 0000020C98CD8960>, < cv2.KeyPoint 0000020C98CD9020>, < cv2.KeyPoint 0000020C98CD8540>, < cv2.KeyPoint 0000020C98CD9050>, < cv2.KeyPoint 0000020C98CD8D80>, < cv2.KeyPoint 0000020C98CD9650>, < cv2.KeyPoint 0000020C96DB04E0>, < cv2.KeyPoint 0000020C970D2100>, < cv2.KeyPoint 0000020C970D03F0>, < cv2.KeyPoint 0000020C98D2BC30>, < cv2.KeyPoint 0000020C98D2ABE0>, < cv2.KeyPoint 0000020C98D2B180>, < cv2.KeyPoint 0000020C98D397A0>, < cv2.KeyPoint 0000020C98D39890>, < cv2.KeyPoint 0000020C98D39830>, < cv2.KeyPoint 0000020C98D398C0>, < cv2.KeyPoint 0000020C98D398F0>, < cv2.KeyPoint 0000020C98D39920>, < cv2.KeyPoint 0000020C98D39950>, < cv2.KeyPoint 0000020C98D39980>, < cv2.KeyPoint 0000020C98D399B0>, < cv2.KeyPoint 0000020C98D399E0>, < cv2.KeyPoint 0000020C98D39A10>, < cv2.Key